In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime

from scipy.stats import chisquare, randint

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

from mpl_toolkits.basemap import Basemap

import warnings
warnings.filterwarnings("ignore")


In [ ]:
# load data
path_start = 'C:/Users/Aino/OneDrive - Aalto University/HY/'
filename = path_start + 'kilpisjarvi_combined_data_10m.pkl'
df_tab = pd.read_pickle(filename)

start = datetime.now()
print(f"Starting processing at: {start}")

In [ ]:
df_tab['month'] = pd.to_datetime(df_tab['timestamp']).dt.month
print((df_tab['month'] == 7).sum())


# drop nan values
columns_with_na = ['kost', 'lamp', 'kpeit', 'kkork', 'turve', 'mmaa', 'mlaji']
df_tab = df_tab.dropna(subset=columns_with_na)


# Rename columns
df_tab.rename(columns={'kost': 'soil_m', 'lamp': 'soil_temp', 'kpeit': 'veg_cov',
                  'kkork': 'veg_heig', 'turve': 'peat', 'mmaa': 'soil_thic',
                  'mlaji': 'soil_type', 'lumip': 'snow_cov'}, inplace=True)


# Display the counts of NaN values
print(df_tab.isna().sum())
print(df_tab.shape)

# Define train and test data
# Function to create train and test split for each month
def train_test_split_by_month(df, test_size=0.33, random_state=42):
    return train_test_split(df, test_size=test_size, random_state=random_state)

# Assuming df_tab is your DataFrame and 'month' is the column containing month information
train_list = []
test_list = []

# Split each month's data separately
for month in [6, 7, 8]:
    df_month = df_tab[df_tab['month'] == month]
    train_month, test_month = train_test_split_by_month(df_month)
    train_list.append(train_month)
    test_list.append(test_month)

# Concatenate all train and test data
train = pd.concat(train_list).reset_index(drop=True)
test = pd.concat(test_list).reset_index(drop=True)

# Check the result
print("Train size:", train.shape)
print("Test size:", test.shape)

# training parameters
train_params_sar = ['VH', 'VV', 'cos_ing', 'cslo', 'aspm', 'aspe']

train_params_env = ['soil_temp', 'veg_cov', 'veg_heig', 'peat',
                'soil_thic', 'NDVI', 'LAI','slope_deg', 'dem',
                'aspect_deg']

train_params_all = ['soil_temp', 'VH', 'VV', 'veg_cov', 'veg_heig', 'peat',
                'soil_thic', 'NDVI', 'LAI', 'cos_ing','slope_deg',
                'aspect_deg', 'cslo', 'aspm', 'aspe']

output_var = 'soil_m'

In [ ]:
'''# plot train and test sets
# combine train and test sets with a marker
train['set'] = 'train'
test['set'] = 'test'

# Combine the datasets
combined = pd.concat([train, test], ignore_index=True)

# Plot the distribution for the 'kost' variable
plt.figure(figsize=(10, 6))

# Use distinct colors and different styles for histograms
sns.histplot(data=combined, x='soil_m', hue='set', kde=True, palette={'train': 'blue', 'test': 'orange'}, element='step', alpha=0.6)

# Add titles and labels
plt.title('Distribution of Soil Moisture in Train and Test Sets')
plt.xlabel('Soil Moisture')
plt.ylabel('Density')

# Make the legend more distinct
plt.legend(title='Dataset', loc='upper right', labels=['Train', 'Test'])

# Save the plot
plt.savefig(path_start + 'ml/distribution_kost.png')


# Create the pair plot
pair_plot = sns.pairplot(df_tab[['soil_m', 'VH', 'VV', 'cos_ing', 'cslo']], diag_kind='kde', markers='o', hue=None)

# Show the plot
for ax in pair_plot.axes.flatten():
    ax.set_xlabel(ax.get_xlabel(), fontsize=16)  # Increase x-axis label size
    ax.set_ylabel(ax.get_ylabel(), fontsize=16)  # Increase y-axis label size
    ax.tick_params(axis='both', which='major', labelsize=14) 
pair_plot.savefig(path_start + 'ml/sar_param_plot.png')


# Scatter plots for selected features
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
sns.scatterplot(data=combined, x='VH', y='VV', hue='set', palette='viridis', alpha=0.7, ax=axes[0, 0])
axes[0, 0].set_title('Scatter Plot of VH vs VV')
axes[0, 0].set_xlim(combined['VH'].min(), combined['VH'].max())
axes[0, 0].set_ylim(combined['VV'].min(), combined['VV'].max())

sns.scatterplot(data=combined, x='cos_ing', y='cslo', hue='set', palette='viridis', alpha=0.7, ax=axes[0, 1])
axes[0, 1].set_title('Scatter Plot of cos_ing vs cslo')
axes[0, 1].set_xlim(combined['cos_ing'].min(), combined['cos_ing'].max())
axes[0, 1].set_ylim(combined['cslo'].min(), combined['cslo'].max())

sns.scatterplot(data=combined, x='NDVI', y='LAI', hue='set', palette='viridis', alpha=0.7, ax=axes[1, 0])
axes[1, 0].set_title('Scatter Plot of NDVI vs LAI')
axes[1, 0].set_xlim(combined['NDVI'].min(), combined['NDVI'].max())
axes[1, 0].set_ylim(combined['LAI'].min(), combined['LAI'].max())

sns.scatterplot(data=combined, x='kpeit', y='turve', hue='set', palette='viridis', alpha=0.7, ax=axes[1, 1])
axes[1, 1].set_title('Scatter Plot of kpeit vs turve')
axes[1, 1].set_xlim(combined['kpeit'].min(), combined['kpeit'].max())
axes[1, 1].set_ylim(combined['turve'].min(), combined['turve'].max())


plt.tight_layout()
plt.savefig(path_start + 'ml/scatter_plots_train_test.png')

# Pair plot for all training parameters
pair_plot_vars = train_params_sar + train_params_env + [output_var, 'set']
sns.pairplot(combined[pair_plot_vars], hue='set', palette='viridis', diag_kind='kde', markers=['o', 's'])
plt.suptitle('Pair Plot of Selected Features in Train and Test Sets', y=1.02)
plt.savefig(path_start + 'ml/pair_plot_train.png')

# Heatmap for correlation matrix
plt.figure(figsize=(14, 10))
correlation_matrix = combined[train_params_all + [output_var]].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='viridis')
plt.title('Correlation Matrix of Training Parameters and Output Variable')
plt.savefig(path_start + 'ml/correlation_train.png')
'''

In [ ]:
# Function to train and evaluate models
def train_evaluate_model(X_train, y_train, X_test, y_test, model, title):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Combine true values and predicted values for trend line calculation
    y_true_combined = np.concatenate([y_train, y_test])
    y_pred_combined = np.concatenate([y_train_pred, y_test_pred])

    # Calculate trend line using linear regression on combined predictions and true values
    trend_coefficients = np.polyfit(y_pred_combined, y_true_combined, 1)
    trend_line = np.poly1d(trend_coefficients)

    # Generate trend line points
    trend_x = np.linspace(min(y_pred_combined), max(y_pred_combined), 100)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.scatter(y_train_pred, y_train, c='red', alpha=0.5, label='Train')
    plt.scatter(y_test_pred, y_test, c='blue', alpha=0.5, label='Test')
    plt.plot(trend_x, trend_line(trend_x), color='black', linestyle='--', linewidth=3, label='Trend Line')
    plt.xlim(0, 100)
    plt.ylim(0, 100)
    plt.tick_params(axis='both', which='major', labelsize=14)  # Increase the tick label size
    plt.grid()
    plt.title(title, fontsize=16)
    plt.ylabel('Measured Soil Moisture (VWC%)', fontsize=14)
    plt.xlabel('Predicted Soil Moisture (VWC%)', fontsize=14)
    plt.legend(fontsize=14)

    # Evaluation metrics
    print(f'{title} Train RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred))}')
    print(f'{title} Test RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
    print(f'{title} Train Corr: {np.corrcoef(y_train, y_train_pred)[0, 1]}')
    print(f'{title} Test Corr: {np.corrcoef(y_test, y_test_pred)[0, 1]}')
    print(f'{title} Train R²: {r2_score(y_train, y_train_pred)}')
    print(f'{title} Test R²: {r2_score(y_test, y_test_pred)}')
    print(f'{title} Train Explained Variance: {explained_variance_score(y_train, y_train_pred)}')
    print(f'{title} Test Explained Variance: {explained_variance_score(y_test, y_test_pred)}')

    plt.tight_layout()
    plt.savefig(path_start + f'ml/{title.replace(" ", "_").lower()}.png')

# Define a function to perform GridSearchCV
def grid_search_cv(model, param_grid, X_train, y_train, model_name):
    grid_search = GridSearchCV(model, param_grid, cv=7, verbose=1, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print(f"Best {model_name} params:", grid_search.best_params_)
    return grid_search.best_estimator_

# Define a function to normalize data
def normalize_data(train_df, test_df, params):
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(train_df[params]), columns=params)
    y_train = train_df[output_var]
    X_test = pd.DataFrame(scaler.transform(test_df[params]), columns=params)
    y_test = test_df[output_var]
    return X_train, y_train, X_test, y_test

# Common model parameters for GridSearchCV
random_seed = 42
np.random.seed(random_seed)

rf_param_grid = {
    'n_estimators': [500, 700, 1000],
    'max_depth': [2, 4, 6],
    'min_samples_split': [10, 14, 17],
    'min_samples_leaf': [10, 13, 15],
    'max_features': [None, 'sqrt', 'log2'],
    'bootstrap': [True],
    'max_samples': [0.7, 0.8, 0.9]
}

gb_param_grid = {
    'learning_rate': [0.001, 0.005, 0.01],
    'n_estimators': [500, 1000, 1500],
    'max_depth': [2, 3],
    'min_samples_split': [7, 10, 12],
    'min_samples_leaf': [4, 6, 8],
    'subsample': [0.7, 0.8],
    'max_features': ['auto', 'sqrt', 'log2'],
    'loss': ['huber', 'squared_error', 'absolute_error']
    #'warm_start': [True]
}


In [ ]:
# Test 1: Model with SAR data
print('TEST 1:')

X_train_sar, y_train_sar, X_test_sar, y_test_sar = normalize_data(train, test, train_params_sar)

rf_best_sar = grid_search_cv(RandomForestRegressor(random_state=random_seed), rf_param_grid, X_train_sar, y_train_sar, 'RF')
gb_best_sar = grid_search_cv(GradientBoostingRegressor(random_state=random_seed), gb_param_grid, X_train_sar, y_train_sar, 'GB')

# Plot and evaluate models
train_evaluate_model(X_train_sar, y_train_sar, X_test_sar, y_test_sar, rf_best_sar, 'RF Best - SAR')
train_evaluate_model(X_train_sar, y_train_sar, X_test_sar, y_test_sar, gb_best_sar, 'GB Best - SAR')


In [ ]:
# Test 2: Model with environmental data
print('TEST 2:')
X_train_env, y_train_env, X_test_env, y_test_env = normalize_data(train, test, train_params_env)

rf_best_env = grid_search_cv(RandomForestRegressor(random_state=random_seed), rf_param_grid, X_train_env, y_train_env, 'RF')
gb_best_env = grid_search_cv(GradientBoostingRegressor(random_state=random_seed), gb_param_grid, X_train_env, y_train_env, 'GB')

# Plot and evaluate models
train_evaluate_model(X_train_env, y_train_env, X_test_env, y_test_env, rf_best_env, 'RF Best - ENV')
train_evaluate_model(X_train_env, y_train_env, X_test_env, y_test_env, gb_best_env, 'GB Best - ENV')


In [ ]:
# Test 3: Model with all data
print('TEST 3:')
X_train_all, y_train_all, X_test_all, y_test_all = normalize_data(train, test, train_params_all)

rf_best_all = grid_search_cv(RandomForestRegressor(random_state=random_seed), rf_param_grid, X_train_all, y_train_all, 'RF')
gb_best_all = grid_search_cv(GradientBoostingRegressor(random_state=random_seed), gb_param_grid, X_train_all, y_train_all, 'GB')

# Plot and evaluate models
train_evaluate_model(X_train_all, y_train_all, X_test_all, y_test_all, rf_best_all, 'RF Best - ALL')
train_evaluate_model(X_train_all, y_train_all, X_test_all, y_test_all, gb_best_all, 'GB Best - ALL')


In [ ]:
# Feature importances for the models
# Random Forest Feature Importance
def plot_feature_importance(rf_model, gb_model, feature_names, title, filename):
    rf_importances = rf_model.feature_importances_
    gb_importances = gb_model.feature_importances_

    rf_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': rf_importances}).sort_values(by='Importance', ascending=False)
    gb_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': gb_importances}).sort_values(by='Importance', ascending=False)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
    fig.suptitle(title, fontsize=20)

    axes[0].barh(rf_importance_df['Feature'], rf_importance_df['Importance'])
    axes[0].invert_yaxis()
    axes[0].set_title('Random Forest Feature Importances', fontsize=16)
    axes[0].set_xlabel('Relative Importance', fontsize=14)
    axes[0].set_xlim([0, 0.5])
    axes[0].tick_params(axis='y', labelsize=13)
    axes[0].tick_params(axis='x', labelsize=13)

    axes[1].barh(gb_importance_df['Feature'], gb_importance_df['Importance'])
    axes[1].invert_yaxis()
    axes[1].set_title('Gradient Boosting Feature Importances', fontsize=16)
    axes[1].set_xlabel('Relative Importance', fontsize=12)
    axes[1].set_xlim([0, 0.5])
    axes[1].tick_params(axis='y', labelsize=13)
    axes[1].tick_params(axis='x', labelsize=13)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    print(f"Saving plot to {filename}")
    plt.savefig(filename)
    plt.close()

# Feature sets
feature_sets = {
    "sar": train_params_sar,
    "env": train_params_env,
    "all": train_params_all
}

# Models
models = {
    "sar": (rf_best_sar, gb_best_sar),
    "env": (rf_best_env, gb_best_env),
    "all": (rf_best_all, gb_best_all)
}

# Plot feature importances for each feature set and model combination
for feature_set_name, (rf_model, gb_model) in models.items():
    features = feature_sets[feature_set_name]
    plot_title = f"Feature Importances - {feature_set_name.upper()}"
    file_name = path_start + f"ml/importance60_{feature_set_name}.png"
    plot_feature_importance(rf_model, gb_model, features, plot_title, file_name)


In [ ]:
# predicting soil moisture
print("Predicting soil moisture...")
# Define function to prepare monthly data
def prepare_monthly_data(monthly_df, scaler, params):
    X_month = pd.DataFrame(scaler.transform(monthly_df[params]), columns=params)
    return X_month
    
# Define function to make maps of the actual soil moisture values

def plot_actual_kost(monthly_df, lat, lon, title, filename, lat_bins=30, lon_bins=30):
    kost_values = monthly_df['soil_m']

    # Create a grid based on latitude and longitude
    lat_edges = np.linspace(lat.min(), lat.max(), lat_bins + 1)
    lon_edges = np.linspace(lon.min(), lon.max(), lon_bins + 1)
    lat_centers = 0.5 * (lat_edges[:-1] + lat_edges[1:])
    lon_centers = 0.5 * (lon_edges[:-1] + lon_edges[1:])

    # Create a 2D histogram to count the number of points in each bin
    grid, _, _ = np.histogram2d(lat, lon, bins=[lat_edges, lon_edges], weights=kost_values)

    # Normalize the grid by the number of points in each bin
    counts, _, _ = np.histogram2d(lat, lon, bins=[lat_edges, lon_edges])
    grid = np.divide(grid, counts, where=counts != 0)
    grid[counts == 0] = np.nan

    # Set up the Basemap
    fig, ax = plt.subplots(figsize=(10, 8))

    # Plot the data on the map
    mesh = ax.pcolormesh(lon_centers, lat_centers, grid.T, shading='auto', cmap='BrBG', vmin=0, vmax=80)

    cbar = plt.colorbar(mesh, ax=ax)
    cbar.set_label('Soil Moisture (VWC%)', fontsize=14)
    plt.title(title, fontsize=16)
    plt.xlabel('Longitude', fontsize=14)
    plt.ylabel('Latitude', fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=14)  # Increase the tick label size
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

def make_predictions(model, X_month, lat, lon, title, filename,  lat_bins=30, lon_bins=30):
    y_month_pred = model.predict(X_month)

    # Create a grid based on latitude and longitude
    lat_edges = np.linspace(lat.min(), lat.max(), lat_bins + 1)
    lon_edges = np.linspace(lon.min(), lon.max(), lon_bins + 1)
    lat_centers = 0.5 * (lat_edges[:-1] + lat_edges[1:])
    lon_centers = 0.5 * (lon_edges[:-1] + lon_edges[1:])

    # Create a 2D histogram to count the number of points in each bin
    grid, _, _ = np.histogram2d(lat, lon, bins=[lat_edges, lon_edges], weights=y_month_pred)
    counts, _, _ = np.histogram2d(lat, lon, bins=[lat_edges, lon_edges])
    grid = np.divide(grid, counts, where=counts != 0)
    grid[counts == 0] = np.nan

    # Set up the Basemap
    fig, ax = plt.subplots(figsize=(10, 8))

    # Plot the data on the map
    mesh = ax.pcolormesh(lon_centers, lat_centers, grid.T, shading='auto', cmap='BrBG', vmin=0, vmax=80)

    cbar = plt.colorbar(mesh, ax=ax)
    cbar.set_label('Soil Moisture (VWC%)', fontsize=14)
    plt.title(title, fontsize=16)
    plt.xlabel('Longitude', fontsize=14)
    plt.ylabel('Latitude', fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=14)  # Increase the tick label size
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()


In [ ]:
# Monthly data
monthly_data_june = df_tab[df_tab['month'] == 6]
monthly_data_july = df_tab[df_tab['month'] == 7]
monthly_data_august = df_tab[df_tab['month'] == 8]

test_june_sar = test[test['month'] == 6][train_params_sar]
test_july_sar = test[test['month'] == 7][train_params_sar]
test_august_sar = test[test['month'] == 8][train_params_sar]

test_june_env = test[test['month'] == 6][train_params_env]
test_july_env = test[test['month'] == 7][train_params_env]
test_august_env = test[test['month'] == 8][train_params_env]

test_june_all = test[test['month'] == 6][train_params_all]
test_july_all = test[test['month'] == 7][train_params_all]
test_august_all = test[test['month'] == 8][train_params_all]

# Normalize the data using the same scaler
scaler_sar = StandardScaler().fit(train[train_params_sar])
scaler_env = StandardScaler().fit(train[train_params_env])
scaler_all = StandardScaler().fit(train[train_params_all])

# Prepare the data for each month
X_test_june_sar = prepare_monthly_data(test_june_sar, scaler_sar, train_params_sar)
X_test_july_sar = prepare_monthly_data(test_july_sar, scaler_sar, train_params_sar)
X_test_august_sar = prepare_monthly_data(test_august_sar, scaler_sar, train_params_sar)

X_test_june_env = prepare_monthly_data(test_june_env, scaler_env, train_params_env)
X_test_july_env = prepare_monthly_data(test_july_env, scaler_env, train_params_env)
X_test_august_env = prepare_monthly_data(test_august_env, scaler_env, train_params_env)

X_test_june_all = prepare_monthly_data(test_june_all, scaler_all, train_params_all)
X_test_july_all = prepare_monthly_data(test_july_all, scaler_all, train_params_all)
X_test_august_all = prepare_monthly_data(test_august_all, scaler_all, train_params_all)


# Make predictions and plot the results

# SAR Data
make_predictions(rf_best_sar, X_test_june_sar, test[test['month'] == 6]['lat'], test[test['month'] == 6]['lon'], 'RF SAR Predictions - June', path_start + 'ml/predictions_rf_sar_june.png')
make_predictions(rf_best_sar, X_test_july_sar, test[test['month'] == 7]['lat'], test[test['month'] == 7]['lon'], 'RF SAR Predictions - July', path_start + 'ml/predictions_rf_sar_july.png')
make_predictions(rf_best_sar, X_test_august_sar, test[test['month'] == 8]['lat'], test[test['month'] == 8]['lon'], 'RF SAR Predictions - August', path_start + 'ml/predictions_rf_sar_august.png')

make_predictions(gb_best_sar, X_test_june_sar, test[test['month'] == 6]['lat'], test[test['month'] == 6]['lon'], 'GB SAR Predictions - June', path_start + 'ml/predictions_gb_sar_june.png')
make_predictions(gb_best_sar, X_test_july_sar, test[test['month'] == 7]['lat'], test[test['month'] == 7]['lon'], 'GB SAR Predictions - July', path_start + 'ml/predictions_gb_sar_july.png')
make_predictions(gb_best_sar, X_test_august_sar, test[test['month'] == 8]['lat'], test[test['month'] == 8]['lon'], 'GB SAR Predictions - August', path_start + 'ml/predictions_gb_sar_august.png')

# Environmental Data
make_predictions(rf_best_env, X_test_june_env, test[test['month'] == 6]['lat'], test[test['month'] == 6]['lon'], 'RF ENV Predictions - June', path_start + 'ml/predictions_rf_env_june.png')
make_predictions(rf_best_env, X_test_july_env, test[test['month'] == 7]['lat'], test[test['month'] == 7]['lon'], 'RF ENV Predictions - July', path_start + 'ml/predictions_rf_env_july.png')
make_predictions(rf_best_env, X_test_august_env, test[test['month'] == 8]['lat'], test[test['month'] == 8]['lon'], 'RF ENV Predictions - August', path_start + 'ml/predictions_rf_env_august.png')

make_predictions(gb_best_env, X_test_june_env, test[test['month'] == 6]['lat'], test[test['month'] == 6]['lon'], 'GB ENV Predictions - June', path_start + 'ml/predictions_gb_env_june.png')
make_predictions(gb_best_env, X_test_july_env, test[test['month'] == 7]['lat'], test[test['month'] == 7]['lon'], 'GB ENV Predictions - July', path_start + 'ml/predictions_gb_env_july.png')
make_predictions(gb_best_env, X_test_august_env, test[test['month'] == 8]['lat'], test[test['month'] == 8]['lon'], 'GB ENV Predictions - August', path_start + 'ml/predictions_gb_env_august.png')

# All Data
make_predictions(rf_best_all, X_test_june_all, test[test['month'] == 6]['lat'], test[test['month'] == 6]['lon'], 'RF ALL Predictions - June', path_start + 'ml/predictions_rf_all_june.png')
make_predictions(rf_best_all, X_test_july_all, test[test['month'] == 7]['lat'], test[test['month'] == 7]['lon'], 'RF ALL Predictions - July', path_start + 'ml/predictions_rf_all_july.png')
make_predictions(rf_best_all, X_test_august_all, test[test['month'] == 8]['lat'], test[test['month'] == 8]['lon'], 'RF ALL Predictions - August', path_start + 'ml/predictions_rf_all_august.png')

make_predictions(gb_best_all, X_test_june_all, test[test['month'] == 6]['lat'], test[test['month'] == 6]['lon'], 'GB ALL Predictions - June', path_start + 'ml/predictions_gb_all_june.png')
make_predictions(gb_best_all, X_test_july_all, test[test['month'] == 7]['lat'], test[test['month'] == 7]['lon'], 'GB ALL Predictions - July', path_start + 'ml/predictions_gb_all_july.png')
make_predictions(gb_best_all, X_test_august_all, test[test['month'] == 8]['lat'], test[test['month'] == 8]['lon'], 'GB ALL Predictions - August', path_start + 'ml/predictions_gb_all_august.png')

# Plot actual kost values
plot_actual_kost(test[test['month'] == 6], test[test['month'] == 6]['lat'], test[test['month'] == 6]['lon'], 'Measured Soil Moisture (kost) - June', path_start + 'ml/test_actual_kost_june.png')
plot_actual_kost(test[test['month'] == 7], test[test['month'] == 7]['lat'], test[test['month'] == 7]['lon'], 'Measured Soil Moisture (kost) - July', path_start + 'ml/test_actual_kost_july.png')
plot_actual_kost(test[test['month'] == 8], test[test['month'] == 8]['lat'], test[test['month'] == 8]['lon'], 'Measured Soil Moisture (kost) - August', path_start + 'ml/test_actual_kost_august.png')

# Processing end
end = datetime.now()
print(f"Finished processing at: {end}")
processing_time = end - start
print(f"Processing took: {processing_time.total_seconds() / 3600} hours")

In [ ]:
def evaluate_predictions(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, r2
    
# Define a function to make predictions and evaluate them
def make_and_evaluate_predictions(model, X_month, y_true, month, model_name, feature_set):
    y_pred = model.predict(X_month)
    mae, mse, rmse, r2 = evaluate_predictions(y_true, y_pred)
    print(f"{model_name} Predictions ({feature_set}) - {month}: MAE={mae}, MSE={mse}, RMSE={rmse}, R2={r2}")
    return mae, mse, rmse, r2

# Define the actual values for evaluation
y_june_actual = test[test['month'] == 6][output_var].values
y_july_actual = test[test['month'] == 7][output_var].values
y_august_actual = test[test['month'] == 8][output_var].values

# Models and feature sets
models = {
    'RF': {'sar': rf_best_sar, 'env': rf_best_env, 'all': rf_best_all},
    'GB': {'sar': gb_best_sar, 'env': gb_best_env, 'all': gb_best_all}
}

feature_sets = {
    'sar': {'june': X_test_june_sar, 'july': X_test_july_sar, 'august': X_test_august_sar},
    'env': {'june': X_test_june_env, 'july': X_test_july_env, 'august': X_test_august_env},
    'all': {'june': X_test_june_all, 'july': X_test_july_all, 'august': X_test_august_all}
}

actual_values = {
    'june': y_june_actual,
    'july': y_july_actual,
    'august': y_august_actual
}

# Evaluate all models and feature sets for each month
for model_name, model_dict in models.items():
    for feature_set, model in model_dict.items():
        for month, X_month in feature_sets[feature_set].items():
            y_true = actual_values[month]
            make_and_evaluate_predictions(model, X_month, y_true, month, model_name, feature_set)


In [ ]:
# For train data (just for comparison)
# Normalize the data using the same scaler
scaler_sar = StandardScaler().fit(train[train_params_sar])
scaler_env = StandardScaler().fit(train[train_params_env])
scaler_all = StandardScaler().fit(train[train_params_all])


train_june_sar = train[train['month'] == 6][train_params_sar]
train_july_sar = train[train['month'] == 7][train_params_sar]
train_august_sar = train[train['month'] == 8][train_params_sar]

train_june_env = train[train['month'] == 6][train_params_env]
train_july_env = train[train['month'] == 7][train_params_env]
train_august_env = train[train['month'] == 8][train_params_env]

train_june_all = train[train['month'] == 6][train_params_all]
train_july_all = train[train['month'] == 7][train_params_all]
train_august_all = train[train['month'] == 8][train_params_all]

# Prepare the data for each month
X_train_june_sar = prepare_monthly_data(train_june_sar, scaler_sar, train_params_sar)
X_train_july_sar = prepare_monthly_data(train_july_sar, scaler_sar, train_params_sar)
X_train_august_sar = prepare_monthly_data(train_august_sar, scaler_sar, train_params_sar)

X_train_june_env = prepare_monthly_data(train_june_env, scaler_env, train_params_env)
X_train_july_env = prepare_monthly_data(train_july_env, scaler_env, train_params_env)
X_train_august_env = prepare_monthly_data(train_august_env, scaler_env, train_params_env)

X_train_june_all = prepare_monthly_data(train_june_all, scaler_all, train_params_all)
X_train_july_all = prepare_monthly_data(train_july_all , scaler_all, train_params_all)
X_train_august_all = prepare_monthly_data(train_august_all, scaler_all, train_params_all)


# Make predictions and plot the results

# SAR Data
make_predictions(rf_best_sar, X_train_june_sar, train[train['month'] == 6]['lat'], train[train['month'] == 6]['lon'], 'RF SAR Predictions - June', path_start + 'ml/predictions_rf_sar_june2.png')
make_predictions(rf_best_sar, X_train_july_sar, train[train['month'] == 7]['lat'], train[train['month'] == 7]['lon'], 'RF SAR Predictions - July', path_start + 'ml/predictions_rf_sar_july2.png')
make_predictions(rf_best_sar, X_train_august_sar, train[train['month'] == 8]['lat'], train[train['month'] == 8]['lon'], 'RF SAR Predictions - August', path_start + 'ml/predictions_rf_sar_august2.png')

make_predictions(gb_best_sar, X_train_june_sar, train[train['month'] == 6]['lat'], train[train['month'] == 6]['lon'], 'GB SAR Predictions - June', path_start + 'ml/predictions_gb_sar_june2.png')
make_predictions(gb_best_sar, X_train_july_sar, train[train['month'] == 7]['lat'], train[train['month'] == 7]['lon'], 'GB SAR Predictions - July', path_start + 'ml/predictions_gb_sar_july2.png')
make_predictions(gb_best_sar, X_train_august_sar, train[train['month'] == 8]['lat'], train[train['month'] == 8]['lon'], 'GB SAR Predictions - August', path_start + 'ml/predictions_gb_sar_august2.png')

# Environmental Data
make_predictions(rf_best_env, X_train_june_env, train[train['month'] == 6]['lat'], train[train['month'] == 6]['lon'], 'RF ENV Predictions - June', path_start + 'ml/predictions_rf_env_june2.png')
make_predictions(rf_best_env, X_train_july_env, train[train['month'] == 7]['lat'], train[train['month'] == 7]['lon'], 'RF ENV Predictions - July', path_start + 'ml/predictions_rf_env_july2.png')
make_predictions(rf_best_env, X_train_august_env, train[train['month'] == 8]['lat'], train[train['month'] == 8]['lon'], 'RF ENV Predictions - August', path_start + 'ml/predictions_rf_env_august2.png')

make_predictions(gb_best_env, X_train_june_env, train[train['month'] == 6]['lat'], train[train['month'] == 6]['lon'], 'GB ENV Predictions - June', path_start + 'ml/predictions_gb_env_june2.png')
make_predictions(gb_best_env, X_train_july_env, train[train['month'] == 7]['lat'], train[train['month'] == 7]['lon'], 'GB ENV Predictions - July', path_start + 'ml/predictions_gb_env_july2.png')
make_predictions(gb_best_env, X_train_august_env, train[train['month'] == 8]['lat'], train[train['month'] == 8]['lon'], 'GB ENV Predictions - August', path_start + 'ml/predictions_gb_env_august2.png')

# All Data
make_predictions(rf_best_all, X_train_june_all, train[train['month'] == 6]['lat'], train[train['month'] == 6]['lon'], 'RF ALL Predictions - June', path_start + 'ml/predictions_rf_all_june2.png')
make_predictions(rf_best_all, X_train_july_all, train[train['month'] == 7]['lat'], train[train['month'] == 7]['lon'], 'RF ALL Predictions - July', path_start + 'ml/predictions_rf_all_july2.png')
make_predictions(rf_best_all, X_train_august_all, train[train['month'] == 8]['lat'], train[train['month'] == 8]['lon'], 'RF ALL Predictions - August', path_start + 'ml/predictions_rf_all_august2.png')

make_predictions(gb_best_all, X_train_june_all, train[train['month'] == 6]['lat'], train[train['month'] == 6]['lon'], 'GB ALL Predictions - June', path_start + 'ml/predictions_gb_all_june2.png')
make_predictions(gb_best_all, X_train_july_all, train[train['month'] == 7]['lat'], train[train['month'] == 7]['lon'], 'GB ALL Predictions - July', path_start + 'ml/predictions_gb_all_july2.png')
make_predictions(gb_best_all, X_train_august_all, train[train['month'] == 8]['lat'], train[train['month'] == 8]['lon'], 'GB ALL Predictions - August', path_start + 'ml/predictions_gb_all_august2.png')

# Plot actual kost values
plot_actual_kost(train[train['month'] == 6], train[train['month'] == 6]['lat'], train[train['month'] == 6]['lon'], 'Measured Soil Moisture (kost) - June', path_start + 'ml/train_actual_kost_june.png')
plot_actual_kost(train[train['month'] == 7], train[train['month'] == 7]['lat'], train[train['month'] == 7]['lon'], 'Measured Soil Moisture (kost) - July', path_start + 'ml/train_actual_kost_july.png')
plot_actual_kost(train[train['month'] == 8], train[train['month'] == 8]['lat'], train[train['month'] == 8]['lon'], 'Measured Soil Moisture (kost) - August', path_start + 'ml/train_actual_kost_august.png')


In [ ]:
# Define the actual values for evaluation
y_june_actual = train[train['month'] == 6][output_var].values
y_july_actual = train[train['month'] == 7][output_var].values
y_august_actual = train[train['month'] == 8][output_var].values

# Models and feature sets
models = {
    'RF': {'sar': rf_best_sar, 'env': rf_best_env, 'all': rf_best_all},
    'GB': {'sar': gb_best_sar, 'env': gb_best_env, 'all': gb_best_all}
}

feature_sets = {
    'sar': {'june': X_train_june_sar, 'july': X_train_july_sar, 'august': X_train_august_sar},
    'env': {'june': X_train_june_env, 'july': X_train_july_env, 'august': X_train_august_env},
    'all': {'june': X_train_june_all, 'july': X_train_july_all, 'august': X_train_august_all}
}

actual_values = {
    'june': y_june_actual,
    'july': y_july_actual,
    'august': y_august_actual
}

# Evaluate all models and feature sets for each month
for model_name, model_dict in models.items():
    for feature_set, model in model_dict.items():
        for month, X_month in feature_sets[feature_set].items():
            y_true = actual_values[month]
            make_and_evaluate_predictions(model, X_month, y_true, month, model_name, feature_set)
